## Import libraries

In [19]:
import numpy as np
from utils.layers import Dense, Conv2D
from utils.losses import Softmax, MeanSquaredError
from utils.optimizers import Optimizer, SGD, SGDMomentum
from utils.activations import Sigmoid, Tanh, Linear, ReLU
from utils.network import NeuralNetwork
from utils.train import Trainer

### Data import and preprocessing

In [20]:
# Import MNIST dataset - Credit: https://github.com/hsjeong5

from urllib import request
import gzip
import pickle

filename = [
["training_images","train-images-idx3-ubyte.gz"],
["test_images","t10k-images-idx3-ubyte.gz"],
["training_labels","train-labels-idx1-ubyte.gz"],
["test_labels","t10k-labels-idx1-ubyte.gz"]
]


def download_mnist():
  base_url = "http://yann.lecun.com/exdb/mnist/"
  for name in filename:
    print("Downloading "+name[1]+"...")
    request.urlretrieve(base_url+name[1], name[1])
  print("Download complete.")


def save_mnist():
  mnist = {}
  for name in filename[:2]:
    with gzip.open(name[1], 'rb') as f:
      mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1,28*28)
  for name in filename[-2:]:
    with gzip.open(name[1], 'rb') as f:
      mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=8)
  with open("mnist.pkl", 'wb') as f:
    pickle.dump(mnist,f)
  print("Save complete.")


def init():
  download_mnist()
  save_mnist()


def load():
  with open("mnist.pkl",'rb') as f:
    mnist = pickle.load(f)
  return mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]

if __name__ == '__main__':
  init()

Download complete.
Save complete.


In [21]:
# Normalize data (mean 0, variance 1)
X_train, y_train, X_test, y_test = load()
X_train, X_test = X_train - np.mean(X_train), X_test - np.mean(X_train)
X_train, X_test = X_train / np.std(X_train), X_test / np.std(X_train)

In [22]:
X_train_conv, X_test_conv = X_train.reshape(-1, 1, 28, 28), X_test.reshape(-1, 1, 28, 28)

In [23]:
# One-hot encoding labels (both train and test)
num_labels = len(y_train)
train_labels = np.zeros((num_labels, 10))
for i in range(num_labels):
  train_labels[i][y_train[i]] = 1

num_labels = len(y_test)
test_labels = np.zeros((num_labels, 10))
for i in range(num_labels):
  test_labels[i][y_test[i]] = 1

In [24]:
# Function for accuracy
def calc_accuracy_model(model, test_set):
        return print(f'''The model validation accuracy is: {np.equal(np.argmax(model.forward(test_set, inference=True), axis=1), y_test).sum() * 100.0 / test_set.shape[0]:.2f}%''')

## Convolutional NN

In [25]:
model = NeuralNetwork(
    layers = [Conv2D(out_channels=16, param_size=5, dropout=0.8, weight_init="glorot", flatten=True, activation=Tanh()),
              Dense(neurons=10, activation=Linear())],
    loss = Softmax(),
    seed = 42
)

In [26]:
trainer = Trainer(model, SGDMomentum(learning_rate=0.1, momentum=0.9))
trainer.fit(X_train_conv, train_labels, X_test_conv, test_labels, epochs = 1, eval_every = 1, seed = 42, batch_size = 60, conv_testing = True);

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)